# Imports

In [ ]:
%matplotlib inline
import SQGfunctions as fn
from SQGfunctions import *
blues = mpl.colormaps['Blues']
greens = mpl.colormaps['Greens']

# Import data

In [ ]:
df = pd.read_csv(r'H:/taryn-phd/sqgData/DRAG_0shift_CMA.csv')
fid = df['FidelityCMA'].to_numpy()
error = df['ErrorCMA'].to_numpy()
lambdas = df['LambdasCMA'].to_numpy()
Eds = df['EdsCMA'].to_numpy()
times = np.linspace(10,50,41)

# Single gate at sweet spot with added quasistatic flux noise

In [ ]:
#collapse operators
c_op1 = []
c_op2 = []

H_sys,evals,evecs = fluxonium_system(sweet_spot,Ej,Ec,El,n_op,phi_op,H_lc,H_i)
Es = np.real(qubit_energies(H_sys,evals,evecs))
omega_drive = (Es[2]-Es[0])
drive_freq = omega_drive #drive frequency at sweet spot

In [ ]:
def gate_pulse_noise(lamb,Ed,gatetime,psi,sweet_spot=np.pi):
    #calculations at sweet spot - assumed parameters
    time = np.linspace(0, gatetime, 101)
    H_sys,evals,evecs = fluxonium_system(np.pi,Ej,Ec,El,n_op,phi_op,H_lc,H_i)
    Es = np.real(qubit_energies(H_sys,evals,evecs))
    omega_drive = (Es[2]-Es[0])
    drive_freq = omega_drive
    alpha = (Es[4] - Es[2]) - (Es[2] - Es[0])
    drive_me, trunc_drive_op = matrix_element(drive_op,H_sys,evals,evecs)
    tgate = gatetime
    
    #calculation of system parameters at "sweet-spot" where noise can be added
    H_sys,evals,evecs = fluxonium_system(sweet_spot,Ej,Ec,El,n_op,phi_op,H_lc,H_i)
    Es = np.real(qubit_energies(H_sys,evals,evecs))
    
    H_total = [Qobj(np.diag(Es)), [Ed*trunc_drive_op, H_drive_TD_no_fp]]
    states = time_evolve_no_fp(H_total, initialState, time, args={'gate_time': tgate, 'lamb': lamb,
                                                    'omega_drive': drive_freq, 
                                                    'alpha': alpha})[1:]
    fidelity = np.abs((expectRho.dag()*states[-1]).full()[0,0])**2
    print('')
    print('lambda',lamb)
    print('drive',Ed)
    print('fidelity',fidelity)
    return fidelity, states

In [ ]:
my_noise_values = np.array([0.0001, 0.001, 0.01])
reps = 50 #iterations of random numbers generated
fidelities = np.zeros((len(my_noise_values), reps, len(times)))
errors = np.zeros((len(my_noise_values), reps, len(times)))

for p, noise in enumerate(tqdm(my_noise_values)):
    for m, j in enumerate(tqdm(range(reps))):
        num = np.random.randn(1)
        xi = noise * num[0] * 2 * np.pi

        sweet_spot = np.pi + xi

        for n, i in enumerate(range(len(times))):
            fids, states = gate_pulse_noise(lambdas[n],Eds[n],times[n],psi=initRho,sweet_spot=sweet_spot)
            fidelities[p,m,n] = fids.real
            errors[p,m,n] = 1 - fids.real

## Average over noise iterations

In [ ]:
fids_compact = np.mean(fidelities, axis=1)
errors_compact = np.mean(errors, axis=1)

# Export data

In [ ]:
data = {'Fidelity_xi0001': fids_compact[0],
        'Fidelity_xi001': fids_compact[1],
        'Fidelity_xi01': fids_compact[2],
        'Error_xi0001': errors_compact[0],
        'Error_xi001': errors_compact[1],
        'Error_xi01': errors_compact[2]}

df = pd.DataFrame(data)
df.to_csv('filepath.csv')

# Plot

In [ ]:
# set number of rows and cols
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

# set plot limits and spacing
fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.3, hspace=0.3)

# fix to single column page width
fig.set_size_inches(3.40457, 2.0)

# font size 8
font = {'size' : 8}
plt.rc('font', **font)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)
ax.tick_params(width=0.5, which='both')
ax.tick_params(length=2.5, which='major')
ax.tick_params(length=1.5, which='minor')

ax.plot(times, errors_compact[2], color=blues(0.8), label = r'$\xi$ = 0.01')
ax.plot(times, errors_compact[1], color=blues(0.6), label = r'$\xi$ = 0.001')
ax.plot(times, errors_compact[0], color=blues(0.4), label = r'$\xi$ = 0.0001')
ax.set_xlabel(r'Gate Time (ns)')
ax.set_ylabel(r'Gate Error')
ax.set_yscale('log')
ax.set_xlim(10,50)
ax.set_ylim(1*10**(-8), 1e0)
ax.legend(loc = 'right',bbox_to_anchor=(0.42, 0.77), handletextpad=0.5, frameon=False)
plt.show()